# Import Packages

In [1]:
from sentence_transformers import SentenceTransformer
from tqdm import tqdm
import requests 
import numpy as np
import pandas as pd

c:\Users\aniel\Documents\github\dev-docker\llm-zoomcamp-aniel-notes\llm-zoomcamp-aniel-notes\.llm-zoomcamp\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


# Q1

In [2]:
model_name = "multi-qa-distilbert-cos-v1"

In [3]:
embedding_model = SentenceTransformer(model_name)

c:\Users\aniel\Documents\github\dev-docker\llm-zoomcamp-aniel-notes\llm-zoomcamp-aniel-notes\.llm-zoomcamp\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\aniel\.cache\huggingface\hub\models--sentence-transformers--multi-qa-distilbert-cos-v1. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [62]:
user_question = "I just discovered the course. Can I still join it?"

In [63]:
vec = embedding_model.encode([user_question])

In [64]:
vec.shape

(1, 768)

In [65]:
np.dot(vec,vec.T)

array([[0.9999999]], dtype=float32)

In [66]:
vec[:,0]

array([0.07822262], dtype=float32)

# Q2

In [9]:
base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/documents-with-ids.json'
docs_url = f'{base_url}/{relative_url}?raw=1'
docs_response = requests.get(docs_url)
documents = docs_response.json()

In [10]:
len(documents)

948

In [11]:
target = "machine-learning-zoomcamp"

selected_docs = [ doc for doc in documents if doc["course"]==target]

In [12]:
selected_docs[0]

{'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 'section': 'General course-related questions',
 'question': 'How do I sign up?',
 'course': 'machine-learning-zoomcamp',
 'id': '0227b872'}

In [13]:

qa_text = [f"{doc['question']} {doc['text']}" for doc in selected_docs]
embeddings = embedding_model.encode(qa_text)

In [26]:
embeddings.shape

(375, 768)

# Q3

In [29]:
embeddings.dot(embeddings.T).shape

(375, 375)

In [32]:
np.trace(np.dot(embeddings,embeddings.T))

375.0

In [33]:
scores = np.dot(embeddings,vec.T)

In [34]:
scores.shape

(375, 1)

In [38]:
scores.max(),scores.min()

(0.65065765, -0.14676371)

# Q4

In [14]:
class VectorSearchEngine():
    def __init__(self, documents, embeddings):
        self.documents = documents
        self.embeddings = embeddings

    def search(self, v_query, num_results=10):
        scores = self.embeddings.dot(v_query)
        idx = np.argsort(-scores[:,0])[:num_results]
        return [self.documents[i] for i in idx]

search_engine = VectorSearchEngine(documents=selected_docs, embeddings=embeddings)
retrieved_docs = search_engine.search(vec.T , num_results=5)

In [15]:


base_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main'
relative_url = '03-vector-search/eval/ground-truth-data.csv'
ground_truth_url = f'{base_url}/{relative_url}?raw=1'

df_ground_truth = pd.read_csv(ground_truth_url)
df_ground_truth = df_ground_truth[df_ground_truth.course == 'machine-learning-zoomcamp']
ground_truth = df_ground_truth.to_dict(orient='records')

In [59]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    
    ith_vec = embedding_model.encode([q['question']])
    results = search_engine.search(ith_vec.T , num_results=5)
    
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

100%|██████████| 1830/1830 [03:15<00:00,  9.35it/s]


In [16]:
def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

In [61]:
hit_rate(relevance_total)

0.9398907103825137

# Q5

In [19]:
len(qa_text)

375

In [20]:
len(selected_docs)

375

In [23]:
qa_text[0],selected_docs[0], embeddings[0].shape

('How do I sign up? Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
 {'text': 'Machine Learning Zoomcamp FAQ\nThe purpose of this document is to capture frequently asked technical questions.\nWe did this for our data engineering course and it worked quite well. Check this document for inspiration on how to structure your questions and answers:\nData Engineering Zoomcamp FAQ\nIn the course GitHub repository there’s a link. Here it is: https://airtable.com/shryxwLd0COOEaqXo\nwork',
  'section': 'General course-related questions',
  'question': 'How do I sign up?',
  'course': 'machine-learning-zoomcamp',
  'id': '0227b872'},
 (768,))

In [25]:
for doc,text_encoded in zip(selected_docs,embeddings):
    doc["text_vector"] = text_encoded

In [27]:
from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200') 

index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
            "id": {"type": "keyword"},
            "text_vector": {"type": "dense_vector", "dims": 768, "index": True, "similarity": "cosine"}
        }
    }
}

index_name = "course-questions"

es_client.indices.delete(index=index_name, ignore_unavailable=True)
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [28]:
from tqdm.auto import tqdm

for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [01:00<00:00, 15.66it/s]


In [67]:
query = {
    "field": "text_vector",
    "query_vector": vec.reshape(-1),
    "k": 5,
    "num_candidates": 10000, 
}

In [68]:
vec.reshape(-1).shape

(768,)

In [69]:
res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "id","course"])
res["hits"]["hits"]

[{'_index': 'course-questions',
  '_id': 'i-ohyJABz5dykPtCQ_qO',
  '_score': 0.82532895,
  '_source': {'question': 'The course has already started. Can I still join it?',
   'course': 'machine-learning-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
   'id': 'ee58a693'}},
 {'_index': 'course-questions',
  '_id': 'juohyJABz5dykPtCRPpC',
  '_score': 0.7358538,
  '_source': {'question': 'I just joined. What should I do next? How can I access course materials?',
   'course': 'machine-learning-zoomcamp',
   'section': 'General course-related questions',
   'text': 'Welcome to the course! Go to the

In [70]:
res["hits"].keys()

dict_keys(['total', 'max_score', 'hits'])

In [71]:
res["hits"]["total"],res["hits"]["max_score"],

({'value': 5, 'relation': 'eq'}, 0.82532895)

In [72]:
len(res["hits"]["hits"])

5

In [73]:
res["hits"]["hits"][0]

{'_index': 'course-questions',
 '_id': 'i-ohyJABz5dykPtCQ_qO',
 '_score': 0.82532895,
 '_source': {'question': 'The course has already started. Can I still join it?',
  'course': 'machine-learning-zoomcamp',
  'section': 'General course-related questions',
  'text': 'Yes, you can. You won’t be able to submit some of the homeworks, but you can still take part in the course.\nIn order to get a certificate, you need to submit 2 out of 3 course projects and review 3 peers’ Projects by the deadline. It means that if you join the course at the end of November and manage to work on two projects, you will still be eligible for a certificate.',
  'id': 'ee58a693'}}

# Q6

In [61]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    
    vec = embedding_model.encode([q['question']])

    query = {
        "field": "text_vector",
        "query_vector": vec.reshape(-1),
        "k": 5,
        "num_candidates": 10000, 
    }
    res = es_client.search(index=index_name, knn=query, source=["text", "section", "question", "id","course"])

    relevance = [d["_source"]['id'] == doc_id for d in res["hits"]["hits"]]
    relevance_total.append(relevance)

100%|██████████| 1830/1830 [03:39<00:00,  8.33it/s]


In [74]:
hit_rate(relevance_total)

0.9398907103825137